<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GTP-2 による文書生成

### 準備

In [ ]:
# ソースからのHuggingface Transformersのインストール
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install -e transformers
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1
# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

### 動作確認

In [ ]:
# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-small")

In [ ]:
# 文書生成
input = tokenizer.encode("みなさん、こんにちは。", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=3)
sentences = tokenizer.batch_decode(output)
for i in sentences:
  print(i)

### ファインチューニング

In [ ]:
%%time
!rm -r output
# ファインチューニングの実行
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-small \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=10 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=2 \
    --per_device_eval_batch_size=2 \
    --fp16 \
    --output_dir=output/ \
    --use_fast_tokenizer=False

In [ ]:
# モデルの準備
model2 = AutoModelForCausalLM.from_pretrained("output/")

# 推論
input = tokenizer.encode("みなさん、こんにちは。", return_tensors="pt")
output2 = model2.generate(input, do_sample=True, max_length=100, num_return_sequences=1)
print(tokenizer.batch_decode(output2))